# Minitrabalho 2 – "Seis graus de separação"

## 1.a)

### Códigos dos Vértices, Arestas e Grafos

In [81]:
# Class Vertice
class Vertex:
    ''' Estrutura de Vértice para um grafo: encapsula um elemento (vertex_id) 
        que é o identificador deste nó.
        
        O elemento (vertex_id) deve ser hashable:  
        - Um objeto hashable é aquele que pode ser utilizado como uma chave num dicionário Python.
        - Isto inclui strings, números, tuplas, etc.
    '''
    
    def __init__(self, vertex_id):
        '''O vértice será inserido no Grafo usando o método insert_vertex(x) que cria um Vertex'''
        self._vertex_id = vertex_id   # Id do vértice (elemento a inserir no grafo)
        self.in_time = None           # Tempo de entrada no vértice (exercício TP3)
        self.out_time = None          # Tempo de saída do vértice (exercício TP3)
        self.status = None            # Marcação de visitado/não visitado (exercício TP3)
        self.parent = None
        self.distance = None
        
    def __hash__(self):
        '''O valor do elemento é usado como hash para o vértice (o elemento deve ser hashable)'''
        return hash(self._vertex_id)  # devolve o hash do elemento

    def __str__(self):
        '''Devolve a representação do objeto vértice em string.'''
        if self.in_time:
            return'v {0}-in({1})-out({2})'.format(self._vertex_id, self.in_time, self.out_time)
        else:
            return'{0}'.format(self._vertex_id)

    def __eq__(self, vertex):
        return self._vertex_id == vertex._vertex_id

    def __lt__(self, vertex):
        return self._vertex_id < vertex._vertex_id
    
    def __le__(self, vertex):
        return self._vertex_id <= vertex._vertex_id
    
    def __gt__(self, vertex):
        return self._vertex_id > vertex._vertex_id
    
    def __ge__(self, vertex):
        return self._vertex_id >= vertex._vertex_id
    
    def vertex_id(self):
        ''' Devolve o elemento guardado neste vértice.'''
        return self._vertex_id

In [3]:
# Class Edge
class Edge:
    ''' Estrutura de Aresta para um Grafo: (origem, destino) e peso '''

    def __init__(self, vertex_1, vertex_2, weight, movie_list):
        self._vertex_1 = vertex_1
        self._vertex_2 = vertex_2
        self._weight = weight
        self._movie_list = movie_list

    def __hash__(self):
        # Função que mapeia a aresta a uma posição no dicionário (hash map)
        return hash( (self._vertex_1, self._vertex_2) )

    def __str__(self):
        ''' Devolve a representação do objeto aresta em string: (origem, destino)w=peso '''
        return'e({0},{1},{2})w={3}'.format(self._vertex_1, self._vertex_2, self._movie_list, self._weight)

    def __eq__(self, other):
        # define igualdade de duas arestas (deve ser consistente com a função hash)
        return self._vertex_1 == other._vertex_1 and self._vertex_2 == other._vertex_2

    def endpoints(self):
        ''' Devolve a tupla (vertex_1, vertex_2) os vértices adjacentes vertex_1 e vertex_2.'''
        return (self._vertex_1, self._vertex_2)

    def cost(self):
        ''' Devolve o peso associado a este arco.'''
        return self._weight
    
    def opposite(self, vertex):
        ''' Indica o vértice oposto ao vertex nesta aresta 
            (apenas se vertex fizer parte da aresta).'''
        if vertex == self._vertex_1:
            return self._vertex_2
        elif vertex == self._vertex_2:
            return self._vertex_1
        else:
            return None
        
    def get_movies(self):
        return self._movie_list

In [4]:
class Graph:
    '''
    Representação de um grafo usando dicionários encadeados (nested dictionaries).
    
    Atributos:
    ----------
    adjancencies: Dicionário externo que associa um vértice (Vertex) a um  
                  mapa de adjacências (dicionario interno)
    vertices: Dicionário auxiliar que associa o id dos vértices do grafo
              a um objeto Vertex (tabela de símbolos).
    n: Número de vértices no Grafo
    m: Número de arestas no Grafo
    
    ----------
'''
    def __init__(self):
        '''Construtor: Cria um grafo vazio (dicionário de _adjancencies).'''
        self._adjancencies = {}  # dicionário que associa o par chave-valor: <Vertex v, Mapa de adjacências de v> 
        self._vertices = {}      # dicionário que associa o par: <id do vértice, objeto Vertex correspondente> 
        self._n = 0              # número de vértices do grafo
        self._m = 0              # número de arestas do grafo
        
    def __str__(self):
        '''Devolve a representação do grafo em string (toString)'''
        if self._n == 0:
            ret = "DAA-Graph: <empty>\n"
        else:
            ret = "DAA-Graph:\n"
            for vertex in self._adjancencies.keys():
                #ret += "vertex-"
                ret += str(vertex) + ": "
                for edge in self.incident_edges(vertex.vertex_id()):
                    ret += str(edge) + "; "
                ret += "\n\n"
        return ret
    
    def is_directed(self):
        '''A classe Graph representa um grafo não orientado.'''
        return False
    
    def order(self):
        '''Ordem de um grafo: a quantidade de vértices no Grafo.'''
        return self._n
    
    def size(self):
        '''Dimensão de um grafo: a quantidade total de arestas do Grafo.'''
        return self._m 
    
    def has_vertex(self, vertex_id):
        '''Verifica se o vértice de id vertex_id está no grafo.'''
        return vertex_id in self._vertices
    
    def has_edge(self, u_id, v_id):
        '''Verifica se a aresta (u_id, v_id) existe no grafo.'''
        if not self.has_vertex(u_id) or not self.has_vertex(v_id):
            return False
        else:
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            return vertex_v in self._adjancencies[vertex_u]
        
    def has_neighbors(self, vertex_id):
        '''Verifica se o vértice de id vertex_id tem vértices adjacentes (vizinhos).'''
        if not self.has_vertex(vertex_id):
            return False
        return self.degree(vertex_id) == 0
    
    def insert_vertex(self, vertex_id):
        '''Insere um novo vértice com o id vertex_id.'''
        if not self.has_vertex(vertex_id):
            vertex = Vertex(vertex_id)
            self._vertices[vertex_id] = vertex  # insere o novo vertice no dicionario de vertices
            self._adjancencies[vertex] = {}     # inicializa o mapa de adjacências deste vértice a vazio
            self._n +=1                         # mais um vértice no grafo

    def insert_edge(self, u_id, v_id, movie,weight=0):
        ''' Cria e insere uma nova aresta entre u_id e v_id com peso weight.
            Se a aresta já existe no grafo, atualiza-se o seu peso.
            Também insere os vértices u_id e v_id, caso não existam.'''
        mlist= [movie]
        if not self.has_vertex(u_id):
            self.insert_vertex(u_id) # insere novo vertex e atualiza n
        if not self.has_vertex(v_id):
            self.insert_vertex(v_id) # insere novo vertex e atualiza n
        if not self.has_edge(u_id, v_id):
            self._m +=1           # atualiza m apenas se a aresta ainda não existir no grafo
        else:
            #print("Existing edge {u_id} and {v_id}. Will only update weight")
            mlist = ((self.get_edge(u_id, v_id)).get_movies()) + mlist
        vertex_u = self._vertices[u_id]
        vertex_v = self._vertices[v_id]
        e = Edge(vertex_u, vertex_v, weight, mlist)    
        self._adjancencies[vertex_u][vertex_v] = e  # coloca v nas adjacências de u
        self._adjancencies[vertex_v][vertex_u] = e  # e u nas adjacências de v (para facilitar a procura de todas as arestas incidentes num vértice)
    
    def incident_edges(self, vertex_id):
        '''Devolve um iterável (gerador) com todas as arestas de um vértice com id vertex_id.'''
        vertex = self._vertices[vertex_id]
        for edge in self._adjancencies[vertex].values(): # para todas as arestas incidentes em v:
            yield edge
    
    def degree(self, vertex_id):
        '''Quantidade de arestas incidentes no vértice v.
        '''
        return len(self._adjancencies[self._vertices[vertex_id]])
    
    def get_vertex(self, vertex_id):
        ''' Devolve o objeto Vertex associado ao elemento vertex_id no grafo
        '''
        return None if not self.has_vertex(vertex_id) else self._vertices[vertex_id] 
    
    def get_edge(self, u_id, v_id):
        ''' Devolve o objeto aresta (Edge) que liga u_id a v_id. 
            Devolve None se não forem adjacentes ou se (um d)os vértices não existirem.'''  
        #if u_id not in self._adjancencies or v_id not in self._adjancencies:
        if not self.has_edge(u_id, v_id):
            return None
        else:
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            return self._adjancencies[vertex_u][vertex_v]
    
    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo (tipo Vertex)'''
        return self._vertices.values()

    def edges(self):
        '''Devolve um iterável sobre todas as arestas do Grafo (sem arestas duplicadas).'''
        seen = {}      # evita a repetição de arestas no grafo não orientado
        for adj_map in self._adjancencies.values():
            for edge in adj_map.values():
                if edge not in seen:
                    yield edge
                seen[edge] = True
    
    def remove_vertex(self, vertex_id):
        '''Remove o vértice com id vertex_id. Se o vértice não existir, não faz nada.'''
        # Passo 1: remover todas as arestas do vértice dado
        # Passo 2: remover todas as arestas incidentes em vertex_id dos mapas de outros vertices
        # Passo 3: remover o vértice com id vertex_id do grafo
        if self.has_vertex(vertex_id):
            lst_copied = list(self.incident_edges(vertex_id)) # copia para a lista para evitar erros de concorrência (remove enquanto itera na lista)
            for edge in lst_copied:
                x, y = edge.endpoints()
                self.remove_edge(x.vertex_id(),y.vertex_id())  # (Passos 1 e 2)
            del self._adjancencies[self._vertices[vertex_id]]  # (Passo 3 - remove do dicionário de adjacências)
            del self._vertices[vertex_id]                      # (Passo 3 - remove do dicionário de vértices)
            self._n -=1
        
    def remove_edge(self, u_id, v_id):
        '''Remove a aresta entre u_id e v_id. Se a aresta não existir, não faz nada.'''
        if  self.has_edge(u_id, v_id):
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            del self._adjancencies[vertex_u][vertex_v]
            if vertex_u != vertex_v:  # laços são removidos apenas uma vez
                del self._adjancencies[vertex_v][vertex_u]
            self._m -=1

### 1.b) Carregar ficheiro

In [5]:
def load_data_to_graph(filename):
    g = Graph()
    file = open(filename,"r", encoding="utf-8")
    content = file.readlines()
    for l in content:
        line = l.strip().split("/")
        for index,actor1 in enumerate(line[1:]):
            for actor2 in line[index+2:]:
                g.insert_edge(actor1,actor2,line[0])
    file.close()
    return g

In [94]:
g = load_data_to_graph("teste.txt")
print(g)

DAA-Graph:
Actor1: e(Actor1,Actor2,["'Film1' (2022)"])w=0; e(Actor1,Actor3,["'Film1' (2022)"])w=0; 

Actor2: e(Actor1,Actor2,["'Film1' (2022)"])w=0; e(Actor2,Actor3,["'Film1' (2022)", "'Film2' (2022)"])w=0; e(Actor2,Actor6,["'Film2' (2022)"])w=0; 

Actor3: e(Actor1,Actor3,["'Film1' (2022)"])w=0; e(Actor2,Actor3,["'Film1' (2022)", "'Film2' (2022)"])w=0; e(Actor3,Actor6,["'Film2' (2022)"])w=0; e(Actor3,Actor14,["'Film4' (2022)"])w=0; e(Actor3,Actor15,["'Film4' (2022)"])w=0; 

Actor6: e(Actor2,Actor6,["'Film2' (2022)"])w=0; e(Actor3,Actor6,["'Film2' (2022)"])w=0; 

Actor7: e(Actor7,Actor8,["'Film3' (2022)"])w=0; e(Actor7,Actor9,["'Film3' (2022)"])w=0; 

Actor8: e(Actor7,Actor8,["'Film3' (2022)"])w=0; e(Actor8,Actor9,["'Film3' (2022)"])w=0; 

Actor9: e(Actor7,Actor9,["'Film3' (2022)"])w=0; e(Actor8,Actor9,["'Film3' (2022)"])w=0; 

Actor14: e(Actor3,Actor14,["'Film4' (2022)"])w=0; e(Actor14,Actor15,["'Film4' (2022)"])w=0; 

Actor15: e(Actor3,Actor15,["'Film4' (2022)"])w=0; e(Actor14,Actor15

## 2) API HollywoodOracle

### 2.a)

### 2.b)

In [7]:
import queue as Queue

In [83]:
class HollywoodOracle:
    def __init__(self, filename):
        self._graph = load_data_to_graph(filename)
        self._center_of_universe = self._graph.get_vertex("Bacon, Kevin")
        self.parent = self.BFS_queue()

    def all_movies(self):
        all_m = set()
        edL = self._graph.edges()

        for e in edL:
            all_m.update(e.get_movies())
        return all_m
    
    def all_actors(self):
        return self._graph.vertices()
        
    def movies_from(self, a):
        m_from = set()
        edL = self._graph.incident_edges(a)

        for e in edL:
            m_from.update(e.get_movies())
        return m_from
        
    def cast_of(self, m):
        l = set()
        edgeL = self._graph.edges()
        for e in edgeL:
            mL = e.get_movies()
            if m in mL:
                a = e.endpoints()
                l.add(a[0]._vertex_id)
                l.add(a[1]._vertex_id)
        return l
        
    def set_center_of_universe(self, x):
        self._center_of_universe = x
        self.parent = self.BFS_queue()

    def number_of_X(self, a):
        return self._graph.get_vertex(a).distance
        
    def path_to_X(self, a):
        return #TODO Usar parent para ser mais eficiente?
        
    def max_number_of_X(self):
        max_n = 0  # max bacon number
        not_related = 0 # number of actors without connection
        for x in self._graph.vertices():
            if x.distance == 'infinite':
                not_related += 1
            elif x.distance > max_n:
                max_n = x.distance
        return (max_n, not_related) 
        
    def count_number_of_X(self, n):
        count = 0
        for x in self._graph.vertices():
            if x.distance == n:
                count += 1
        return count
        
    def average_number_of_X(self):
        sum = 0
        i = 0
        for x in self._graph.vertices():
            if x.distance != 'infinite':
                sum += x.distance
                i += 1 
        return sum/i
    
    def BFS_queue(self):
        for x in self._graph.vertices():
            x.status = False
            x.distance = 'infinite'
        queue = Queue.Queue()
        parent = {}
        s = self._center_of_universe 
        queue.put(s)
        s.status = True
        s.distance = 0
        while not queue.empty():
            v = queue.get()
            inc_edges = self._graph.incident_edges(v._vertex_id)
            for e in inc_edges:
                w = e.opposite(v)
                if not w.status:
                    queue.put(w)
                    parent[w] = v
                    w.distance = v.distance + 1
                    w.status = True
        return parent

In [84]:
# Teste Inicializacao
h = HollywoodOracle("small_dataset_utf8.txt")

In [85]:
# Teste number_of_x(a)
h.number_of_X("Watson, Emma (II)")

2

In [86]:
# Teste all_actors()
t = h.all_actors()
for l in t:
  print(l)

Fitz-Gerald, Lewis
Steele, Rob (I)
Wilson, Frank (II)
Tingwell, Charles 'Bud'
Cassell, Alan (I)
Rodger, Ron
Knez, Bruno
Woodward, Edward
Cisse, Halifa
Quin, Don
Kiefel, Russell
Meagher, Ray
Procanin, Michael
Bernard, Hank
Gray, Ian (I)
Brown, Bryan (I)
Ball, Ray (I)
Mullinar, Rod
Donovan, Terence (I)
Ball, Vincent (I)
Pfitzner, John
Currer, Norman
Thompson, Jack (I)
Nicholls, Jon
Haywood, Chris (I)
Smith, Chris (I)
Mann, Trevor (I)
Henderson, Dick (II)
Lovett, Alan
Bell, Wayne (I)
Waters, John (III)
Osborn, Peter
Peterson, Ron
Cornish, Bridget
Horseman, Sylvia
Seidel, Nellie
West, Barbara
Radford, Elspeth
Reed, Maria
Erskine, Ria
Dick, Judy
Walton, Laurie (I)
Gage, Kevin
Hahn, Archie
Feldman, Corey
Gordon, Gale
Drier, Moosie
Theodore, Brother
Katt, Nicky
Miller, Dick (I)
Hanks, Tom
Dern, Bruce
Turner, Arnold F.
Howard, Rance
Ducommun, Rick
Danziger, Cory
Ajaye, Franklyn
Scott, Carey
Kramer, Jeffrey (I)
Olsen, Dana (I)
Gains, Courtney
Picardo, Robert
Hays, Gary
Davis, Sonny Carl
Gibson,

In [87]:
# Teste all_movies()
t = h.all_movies()
for l in t:
  print(l)

Wizard, The (1989)
Shakespeare in Love (1998)
American Pie 2 (2001)
Videodrome (1983)
Jerk, The (1979)
Yi yi (2000)
Avalon (1990)
Rabid (1977)
Octopussy (1983)
Enemy Below, The (1957)
Hud (1963)
Slacker (1991)
Idle Hands (1999)
Oscar (1991)
Hauru no ugoku shiro (2004)
Benny & Joon (1993)
Down with Love (2003)
In the Army Now (1994)
Daddy Day Care (2003)
Thirteen (2003)
Manon des sources (1986)
Assignment, The (1997)
Daylight (1996)
Like Mike (2002)
Heart and Souls (1993)
Fire in the Sky (1993)
F X (1986)
Trzy kolory: Bialy (1994)
Johnny Guitar (1954)
Tango & Cash (1989)
Never Say Never Again (1983)
Human Stain, The (2003)
Man cheng jin dai huang jin jia (2006)
Office Space (1999)
Take the Money and Run (1969)
Annie (1982)
Girlfight (2000)
Swimming Pool (2003)
Easy Rider (1969)
My Fair Lady (1964)
Daredevil (2003)
Addams Family, The (1991)
Roxanne (1987)
Monster's Ball (2001)
Airheads (1994)
Man with the Golden Gun, The (1974)
Kiss the Girls (1997)
Running Scared (1986)
Garfield: A Tail

In [88]:
# Teste movies_from(a)
t = h.movies_from("Watson, Emma (II)")
for l in t:
    print(l)

Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Sorcerer's Stone (2001)
Harry Potter and the Goblet of Fire (2005)


In [89]:
# Teste max_number_of_X()
h.max_number_of_X()

(6, 603)

In [90]:
# Teste count_number_of_X(n)
h.count_number_of_X(1)

1325

In [91]:
# Teste average_number_of_X()
"{:.2f}".format(h.average_number_of_X())

'2.39'

In [93]:
# Teste cast_of(m)
t = h.cast_of("Harry Potter and the Sorcerer's Stone (2001)")
for l in t:
    print(l)

Hart, Ian
Rickman, Alan
Felton, Tom
Biggerstaff, Sean
Triplets, Saunders
Davis, Warwick (I)
Somerville, Geraldine
Shaw, Fiona
Columbus, Eleanor
Bradley, David (IV)
Tabor, Danielle
Herdman, Josh
Griffiths, Richard (I)
Theakston, Will
Fisher-Becker, Simon
Borowiecki, Ben
Young, Nina
Phillips, Leslie
Taylor, Harry (I)
Rawlins, Adrian
Lewis, Matthew (III)
Fern, Scott
Phelps, Oliver
Watson, Emma (II)
Murray, Devon
Sutherland, Leilah
Harris, Richard (I)
Cleese, John
Bayler, Terence
Grint, Rupert
Holmes, David (VII)
Youngblood, Luke
Melling, Harry
Wright, Bonnie
Enoch, Alfie
Smith, Maggie (I)
Radcliffe, Daniel
Phelps, James (I)
Fearon, Ray
Wanamaker, Zoë
Waylett, Jamie
Rankin, Chris
Troyer, Verne
Walters, Julie
Bremmer, Richard
Coltrane, Robbie
Deadman, Derek
Southern, Jean (I)
Spriggs, Elizabeth (I)
Hurt, John
Dale, Emily


## 3)Testes

### 3.1)

In [ ]:
h = HollywoodOracle("large_dataset_utf8.txt")

In [71]:
def histogram():      ##TODO Falta o gráfico
    im = h.max_number_of_X()[0]
    for a in range(im):
        print("Bacon=" + str(a) + "  n=" + str(h.count_number_of_X(a)))

In [72]:
histogram()

Bacon=0  number=1
Bacon=1  number=2249
Bacon=2  number=218085
Bacon=3  number=561132
Bacon=4  number=111180
Bacon=5  number=7906
Bacon=6  number=903
Bacon=7  number=100


### 3.2)

In [ ]:
#h = HollywoodOracle("large_dataset_utf8.txt")

In [74]:
import csv
def top20_average_X():   ##TODO Falta o gráfico
    csv_file = open("top20imbd.csv","r", encoding="utf-8")
    csv_reader = csv.reader(csv_file, delimiter=';')
    for row in csv_reader:
        h.set_center_of_universe(h._graph.get_vertex(row[1]))
        print(row[1] + "  Average_Number:" + str("{:.2f}".format(h.average_number_of_X())))
    csv_file.close()

In [75]:
top20_average_X()

Depp, Johnny  Average Number:2.87
Damon, Matt  Average Number:2.91
Clooney, George  Average Number:2.88
Jolie, Angelina  Average Number:2.96
Blanchett, Cate  Average Number:2.98


KeyboardInterrupt: 

### 3.3)

In [ ]:
h = HollywoodOracle("small_dataset_utf8.txt")

In [ ]:
import random as rnd
def random1000_average_x():
    return    #TODO